In [1]:
import cv2
import numpy as np
from pathlib import Path
from EvEye.utils.visualization.visualization import *
import matplotlib.pyplot as plt
from natsort import natsorted
from tqdm import tqdm

In [2]:
data_root_path = Path("/mnt/data2T/junyuan/eye-tracking/outputs/EventMasks")
users_path = natsorted(list(data_root_path.glob("*")))
label_root_path = Path("/mnt/data2T/junyuan/eye-tracking/outputs/EllipseLabels")

In [3]:
def get_masks(root_path):
    masks = []
    for file in root_path.iterdir():
        if file.suffix == ".png":
            if file.stem.split("_")[-1] == "mask":
                masks.append(file)
    return natsorted(masks)


def get_ellipse(mask):
    raw_image = load_image(str(mask))[0]
    gray_image = cv2.cvtColor(raw_image, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(
        gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )
    if len(contours) == 0:
        return None
    elif len(contours) == 1:
        if len(contours[0]) < 5:
            return None  # 如果轮廓点少于5个，返回None
        ellipse = cv2.fitEllipse(contours[0])
        return ellipse
    else:
        areas = [cv2.contourArea(c) for c in contours]
        max_index = np.argmax(areas)
        if len(contours[max_index]) < 5:
            return None  # 如果最大的轮廓点少于5个，返回None
        ellipse = cv2.fitEllipse(contours[max_index])
        return ellipse

In [4]:
for user_path in tqdm(users_path):
    file_name = user_path.stem
    with open(label_root_path / f"{file_name}.txt", "w") as f:
        masks = get_masks(user_path)
        for mask in masks:
            timestamp = mask.stem.split("_")[1]
            ellipse = get_ellipse(mask)
            if ellipse is None:
                f.write(f"{timestamp} 0 0 0 0 0\n")
            elif ellipse is not None:
                f.write(
                    f"{timestamp} {ellipse[0][0]:.2f} {ellipse[0][1]:.2f} {ellipse[1][0]:.2f} {ellipse[1][1]:.2f} {ellipse[2]:.2f}\n"
                )

100%|██████████| 364/364 [04:03<00:00,  1.49it/s]
